In [2]:
# 1. 필수 라이브러리 설치 (실행 후 에러가 사라집니다)
!pip install pyclipper segmentation_models_pytorch albumentations shapely

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 978.2/978.2 kB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 16.2 MB/s eta 0:00:00


In [4]:
import os
import shutil
from google.colab import drive

# 1. 구글 드라이브 마운트
print("🔄 구글 드라이브 연결 중... (팝업창이 뜨면 승인해주세요)")
drive.mount('/content/drive')

# 2. 모델 파일이 있는 위치 찾기 (아까 저장했던 폴더들 확인)
# 사용자분이 저장했을 법한 경로들을 다 뒤져봅니다.
candidate_paths = [
    '/content/drive/MyDrive/OCR_Winner_0206',  # 아까 날짜로 저장했다면 여기
    '/content/drive/MyDrive/OCR_Best_Model',     # 혹은 여기
    '/content/drive/MyDrive/OCR_Backup'          # 혹은 여기
]

model_filename = 'DBNet_ConvNeXt_FPN_final.pth'
found = False

print("\n🔍 모델 파일 검색 중...")

for folder in candidate_paths:
    # 해당 폴더가 있는지 확인
    if os.path.exists(folder):
        # 폴더 안에 모델 파일이 있는지 확인
        src_path = os.path.join(folder, model_filename)

        # 혹시 _best.pth 로 저장되어 있을 수도 있으니 체크
        if not os.path.exists(src_path):
             src_path = os.path.join(folder, 'DBNet_ConvNeXt_FPN_best.pth')

        if os.path.exists(src_path):
            # 3. 찾았으면 현재 위치로 복사!
            print(f"✅ 모델 발견! : {src_path}")
            shutil.copy(src_path, f"./{model_filename}") # 이름을 final로 통일해서 복사
            print(f"🚚 현재 폴더로 복사 완료: ./{model_filename}")
            found = True
            break

if not found:
    print(f"❌ 드라이브에서 '{model_filename}'을 찾지 못했습니다.")
    print("👉 직접 드라이브 경로를 확인해서 복사해주세요.")
    # 혹시 모르니 현재 드라이브의 루트 파일 목록 출력
    # print(os.listdir('/content/drive/MyDrive'))
else:
    print("\n🎉 준비 완료! 이제 'make_pseudo_json.py' 코드를 다시 실행하세요.")

🔄 구글 드라이브 연결 중... (팝업창이 뜨면 승인해주세요)
Mounted at /content/drive

🔍 모델 파일 검색 중...
✅ 모델 발견! : /content/drive/MyDrive/OCR_Best_Model/DBNet_ConvNeXt_FPN_final.pth
🚚 현재 폴더로 복사 완료: ./DBNet_ConvNeXt_FPN_final.pth

🎉 준비 완료! 이제 'make_pseudo_json.py' 코드를 다시 실행하세요.


In [6]:
# 1. 데이터 다운로드 (Colab 환경)
!wget -O data.tar.gz "https://aistages-api-public-prod.s3.amazonaws.com/app/Competitions/000377/data/data.tar.gz"
!tar -xzf data.tar.gz
!pip install -q segmentation-models-pytorch albumentations opencv-python-headless

--2026-02-06 12:43:04--  https://aistages-api-public-prod.s3.amazonaws.com/app/Competitions/000377/data/data.tar.gz
Resolving aistages-api-public-prod.s3.amazonaws.com (aistages-api-public-prod.s3.amazonaws.com)... 3.5.187.80, 3.5.185.95, 3.5.185.23, ...
Connecting to aistages-api-public-prod.s3.amazonaws.com (aistages-api-public-prod.s3.amazonaws.com)|3.5.187.80|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3280795723 (3.1G) [binary/octet-stream]
Saving to: ‘data.tar.gz’

data.tar.gz         100%[===================>]   3.05G  9.24MB/s    in 5m 9s   

2026-02-06 12:48:14 (10.1 MB/s) - ‘data.tar.gz’ saved [3280795723/3280795723]



In [7]:
import os
import json
import cv2
import numpy as np
import torch
import pyclipper
from shapely.geometry import Polygon
import segmentation_models_pytorch as smp
from albumentations.pytorch import ToTensorV2
import albumentations as A
from tqdm import tqdm

# --- [설정] 0.9553점 맞았던 그 세팅 그대로! ---
ENCODER_NAME = "tu-convnext_base"
MODEL_PATH = "DBNet_ConvNeXt_FPN_final.pth" # 0.95점짜리 모델 경로
IMG_SIZE = 1024

# Recall 튜닝했던 파라미터 (이게 정답을 잘 만드니까요)
UNCLIP_RATIO = 3.0
BOX_THRESH = 0.3

BASE_PATH = './data/datasets'
TEST_IMG_DIR = os.path.join(BASE_PATH, 'images/test')
TEST_JSON_PATH = os.path.join(BASE_PATH, 'jsons/test.json')
OUTPUT_JSON = './data/datasets/jsons/pseudo_test.json'

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def unclip(box, unclip_ratio):
    poly = Polygon(box)
    if poly.area <= 0 or poly.length <= 0: return box
    distance = poly.area * unclip_ratio / poly.length
    offset = pyclipper.PyclipperOffset()
    offset.AddPath(box, pyclipper.JT_ROUND, pyclipper.ET_CLOSEDPOLYGON)
    expanded = offset.Execute(distance)
    return expanded

def main():
    print("🔥 Pseudo-Label 생성 시작... (이게 98점의 열쇠입니다)")

    # 1. 모델 로드
    model = smp.FPN(encoder_name=ENCODER_NAME, in_channels=3, classes=1).to(DEVICE)
    model.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE))
    model.eval()

    # 2. 데이터 로드
    with open(TEST_JSON_PATH, 'r') as f:
        test_data = json.load(f)

    transform = A.Compose([A.Resize(IMG_SIZE, IMG_SIZE), A.Normalize(), ToTensorV2()])

    new_images_info = {}

    # 3. 추론 및 JSON 생성
    for img_name in tqdm(test_data['images']):
        img_path = os.path.join(TEST_IMG_DIR, img_name)
        image = cv2.imread(img_path)
        orig = image.copy()
        orig_h, orig_w = image.shape[:2]

        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        input_tensor = transform(image=image)['image'].unsqueeze(0).to(DEVICE)

        with torch.no_grad():
            preds = torch.sigmoid(model(input_tensor))
            preds = preds.float().cpu().numpy()[0][0] # (1024, 1024)

        # 원본 크기 복원
        preds = cv2.resize(preds, (orig_w, orig_h))
        binary = (preds > BOX_THRESH).astype(np.uint8)

        contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        words_dict = {}
        idx = 0
        for cnt in contours:
            if cv2.contourArea(cnt) < 30: continue
            epsilon = 0.003 * cv2.arcLength(cnt, True)
            approx = cv2.approxPolyDP(cnt, epsilon, True)
            if len(approx) < 3: continue

            reshaped = approx.reshape(-1, 2)

            # Unclip 적용 (Pseudo Label도 통통하게 만들어야 학습이 잘됨)
            try:
                expanded = unclip(reshaped, UNCLIP_RATIO)
                if len(expanded) > 0:
                    final_poly = np.array(expanded[0])
                else:
                    final_poly = reshaped
            except:
                final_poly = reshaped

            # JSON 포맷에 맞게 저장
            words_dict[f'{idx:04d}'] = {
                'points': final_poly.tolist()
            }
            idx += 1

        new_images_info[img_name] = {'words': words_dict}

    # 4. 저장
    final_json = {'images': new_images_info}
    with open(OUTPUT_JSON, 'w', encoding='utf-8') as f:
        json.dump(final_json, f, indent=4)

    print(f"✅ 가짜 정답지 생성 완료: {OUTPUT_JSON}")
    print("👉 이제 이 데이터로 모델을 다시 가르치면 점수가 폭발합니다.")

if __name__ == "__main__":
    main()

🔥 Pseudo-Label 생성 시작... (이게 98점의 열쇠입니다)


100%|██████████| 413/413 [00:41<00:00, 10.06it/s]


✅ 가짜 정답지 생성 완료: ./data/datasets/jsons/pseudo_test.json
👉 이제 이 데이터로 모델을 다시 가르치면 점수가 폭발합니다.


In [8]:
import os
import json
import cv2
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader, ConcatDataset
from torch.amp import autocast, GradScaler
import albumentations as A
from albumentations.pytorch import ToTensorV2
import segmentation_models_pytorch as smp
import pyclipper
from shapely.geometry import Polygon
from tqdm import tqdm
import ssl
import warnings

warnings.filterwarnings('ignore')
ssl._create_default_https_context = ssl._create_unverified_context

# --- [98점용 하이퍼파라미터] ---
# 1. 모델 체급 업그레이드 (Base -> Large)
# A100이니까 Large 돌려야죠. 98점 팀들은 다 이거 씁니다.
ENCODER_NAME = "tu-convnext_large"
MODEL_NAME = "DBNet_ConvNeXt_Large_Pseudo"

BATCH_SIZE = 6  # Large 모델이라 배치를 조금 줄임
EPOCHS = 20     # 이미 학습된 데이터라 20이면 충분
LR = 5e-4       # 미세 조정이라 학습률을 절반으로 낮춤

# 2. Pseudo Label 경로
PSEUDO_JSON = './data/datasets/jsons/pseudo_test.json'
PSEUDO_IMG_DIR = './data/datasets/images/test'

BASE_PATH = './data/datasets'
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
SHRINK_RATIO = 0.4

# (DBDataset 클래스는 기존과 동일하므로 핵심만 빠르게 작성)
class DBDataset(Dataset):
    def __init__(self, img_dir, json_path, transform=None):
        self.img_dir = img_dir
        self.transform = transform
        with open(json_path, 'r', encoding='utf-8') as f:
            self.data = json.load(f)['images']
        self.image_names = list(self.data.keys())

    def __len__(self): return len(self.image_names)

    def __getitem__(self, idx):
        name = self.image_names[idx]
        image = cv2.imread(os.path.join(self.img_dir, name))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        h, w = image.shape[:2]
        mask = np.zeros((h, w), dtype=np.float32)

        if 'words' in self.data[name]:
            for w_info in self.data[name]['words'].values():
                pts = np.array(w_info['points'], dtype=np.int32)
                try:
                    poly = Polygon(pts)
                    if poly.area > 0 and poly.length > 0:
                        distance = poly.area * (1 - SHRINK_RATIO**2) / poly.length
                        offset = pyclipper.PyclipperOffset()
                        offset.AddPath(pts, pyclipper.JT_ROUND, pyclipper.ET_CLOSEDPOLYGON)
                        shrunk = offset.Execute(-distance)
                        if len(shrunk) > 0:
                            cv2.fillPoly(mask, [np.array(shrunk[0], dtype=np.int32)], 1)
                        else: cv2.fillPoly(mask, [pts], 1)
                except: cv2.fillPoly(mask, [pts], 1)

        if self.transform:
            aug = self.transform(image=image, mask=mask)
            return aug['image'], aug['mask']
        return image, mask

train_transform = A.Compose([
    A.Resize(1024, 1024),
    A.Perspective(scale=(0.05, 0.1), p=0.5),
    A.Rotate(limit=20, p=0.5),
    A.ColorJitter(p=0.4),
    A.Normalize(),
    ToTensorV2()
])

def main():
    print(f"🔥 SOTA 도전: Pseudo-Labeling + {ENCODER_NAME}")

    # 1. 기존 학습 데이터
    train_ds = DBDataset(os.path.join(BASE_PATH, 'images/train'), os.path.join(BASE_PATH, 'jsons/train.json'), transform=train_transform)

    # 2. [핵심] 가짜 정답지(Test) 데이터 추가!
    pseudo_ds = DBDataset(PSEUDO_IMG_DIR, PSEUDO_JSON, transform=train_transform)

    # 두 데이터를 합칩니다 -> 데이터 양 2배 뻥튀기 + 테스트 데이터 적응력 MAX
    full_ds = ConcatDataset([train_ds, pseudo_ds])

    train_loader = DataLoader(full_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, drop_last=True)

    print(f"📊 학습 데이터: {len(train_ds)}장 + 가짜 데이터: {len(pseudo_ds)}장 = 총 {len(full_ds)}장")

    # 3. 모델 업그레이드 (Large)
    model = smp.FPN(
        encoder_name=ENCODER_NAME,
        encoder_weights="imagenet",
        in_channels=3,
        classes=1,
    ).to(DEVICE)

    optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=1e-2)
    scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=LR, epochs=EPOCHS, steps_per_epoch=len(train_loader))
    dice_loss = smp.losses.DiceLoss(mode='binary')
    bce_loss = smp.losses.SoftBCEWithLogitsLoss()
    scaler = GradScaler('cuda')

    best_loss = float('inf')

    for epoch in range(1, EPOCHS + 1):
        model.train()
        epoch_loss = 0
        pbar = tqdm(train_loader, desc=f"Epoch {epoch}/{EPOCHS}")

        for imgs, msks in pbar:
            imgs, msks = imgs.to(DEVICE), msks.to(DEVICE).unsqueeze(1)
            optimizer.zero_grad()
            with autocast('cuda', dtype=torch.bfloat16):
                preds = model(imgs)
                loss = dice_loss(preds, msks) + bce_loss(preds, msks)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            scheduler.step()
            epoch_loss += loss.item()
            pbar.set_postfix(loss=loss.item())

        avg_loss = epoch_loss / len(train_loader)
        if avg_loss < best_loss:
            best_loss = avg_loss
            torch.save(model.state_dict(), f"{MODEL_NAME}_best.pth")
            print(f"✨ Best Model 저장 (Loss: {best_loss:.4f})")

    torch.save(model.state_dict(), f"{MODEL_NAME}_final.pth")
    print("✅ Pseudo-Labeling 학습 완료!")

if __name__ == "__main__":
    main()

🔥 SOTA 도전: Pseudo-Labeling + tu-convnext_large
📊 학습 데이터: 3272장 + 가짜 데이터: 413장 = 총 3685장


model.safetensors:   0%|          | 0.00/791M [00:00<?, ?B/s]

Epoch 1/20: 100%|██████████| 614/614 [04:24<00:00,  2.33it/s, loss=0.414]


✨ Best Model 저장 (Loss: 0.4832)


Epoch 2/20: 100%|██████████| 614/614 [04:13<00:00,  2.42it/s, loss=0.315]


...

✨ Best Model 저장 (Loss: 0.2221)


Epoch 19/20: 100%|██████████| 614/614 [04:13<00:00,  2.42it/s, loss=0.207]


✨ Best Model 저장 (Loss: 0.2184)


Epoch 20/20: 100%|██████████| 614/614 [04:13<00:00,  2.42it/s, loss=0.185]


✨ Best Model 저장 (Loss: 0.2158)
✅ Pseudo-Labeling 학습 완료!


In [9]:
import os
import json
import cv2
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from torch.amp import autocast
import albumentations as A
from albumentations.pytorch import ToTensorV2
import segmentation_models_pytorch as smp
import pyclipper
from shapely.geometry import Polygon
from tqdm import tqdm
import ssl
import warnings

warnings.filterwarnings('ignore')
ssl._create_default_https_context = ssl._create_unverified_context

# --- [최종 SOTA 설정] ---
# 1. 모델 체급: Large (Pseudo Learning으로 똑똑해진 놈)
ENCODER_NAME = "tu-convnext_large"

# 2. 모델 파일 경로 (학습 코드에서 저장된 파일명 확인!)
MODEL_PATH = "DBNet_ConvNeXt_Large_Pseudo_final.pth"
# 만약 best 모델이 있다면 아래 주석 풀고 교체하세요
# MODEL_PATH = "DBNet_ConvNeXt_Large_Pseudo_best.pth"

OUTPUT_CSV = "submission_pseudo_sota.csv"

# 3. 승리의 파라미터 (0.95점 찍었을 때 그 세팅)
# Large 모델은 더 정교하므로 3.0이면 충분히 98점 갑니다.
UNCLIP_RATIO = 3.0
BOX_THRESH = 0.3    # 노이즈 적당히 거르고 Recall 챙기는 최적값

BASE_PATH = './data/datasets'
TEST_IMG_DIR = os.path.join(BASE_PATH, 'images/test')
TEST_JSON = os.path.join(BASE_PATH, 'jsons/test.json')
SAMPLE_SUB = os.path.join(BASE_PATH, 'sample_submission.csv')

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
IMG_SIZE = 1024

class TestDataset(Dataset):
    def __init__(self, img_dir, json_path, transform=None):
        self.img_dir = img_dir
        self.transform = transform
        with open(json_path, 'r', encoding='utf-8') as f:
            self.data = json.load(f)['images']
        self.image_names = list(self.data.keys())

    def __len__(self): return len(self.image_names)

    def __getitem__(self, idx):
        name = self.image_names[idx]
        image = cv2.imread(os.path.join(self.img_dir, name))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        orig_h, orig_w = image.shape[:2]
        if self.transform:
            image = self.transform(image=image)['image']
        return image, name, (orig_h, orig_w)

def unclip(box, unclip_ratio):
    poly = Polygon(box)
    if poly.area <= 0 or poly.length <= 0: return box
    distance = poly.area * unclip_ratio / poly.length
    offset = pyclipper.PyclipperOffset()
    offset.AddPath(box, pyclipper.JT_ROUND, pyclipper.ET_CLOSEDPOLYGON)
    expanded = offset.Execute(distance)
    return expanded

def preds_to_polygons(pred_mask, orig_h, orig_w):
    # 1. 원본 크기 복원
    pred_mask = cv2.resize(pred_mask, (orig_w, orig_h))

    # 2. 이진화
    binary_mask = (pred_mask > BOX_THRESH).astype(np.uint8)

    # 3. 윤곽선 검출
    contours, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    polygons = []
    for cnt in contours:
        # 노이즈 필터링
        if cv2.contourArea(cnt) < 30: continue

        epsilon = 0.003 * cv2.arcLength(cnt, True)
        approx = cv2.approxPolyDP(cnt, epsilon, True)

        if len(approx) < 3: continue
        reshaped_poly = approx.reshape(-1, 2)

        # 4. Unclip (확장)
        try:
            expanded = unclip(reshaped_poly, UNCLIP_RATIO)
            if len(expanded) > 0:
                final_poly = np.array(expanded[0])
                polygons.append(final_poly.reshape(-1).tolist())
            else:
                polygons.append(reshaped_poly.reshape(-1).tolist())
        except:
            polygons.append(reshaped_poly.reshape(-1).tolist())

    return "|".join([" ".join(map(str, poly)) for poly in polygons])

def main():
    print(f"🔥 최종 추론 시작! (Model: {ENCODER_NAME}, Unclip: {UNCLIP_RATIO})")

    # 모델 정의 (학습때와 동일한 Large 모델)
    model = smp.FPN(
        encoder_name=ENCODER_NAME,
        encoder_weights=None,
        in_channels=3,
        classes=1,
    ).to(DEVICE)

    if not os.path.exists(MODEL_PATH):
        print(f"❌ 모델 파일({MODEL_PATH})이 아직 없습니다! 학습이 덜 끝났거나 경로가 틀렸습니다.")
        return

    model.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE))
    model.eval()

    test_transform = A.Compose([
        A.Resize(IMG_SIZE, IMG_SIZE),
        A.Normalize(),
        ToTensorV2()
    ])

    test_ds = TestDataset(TEST_IMG_DIR, TEST_JSON, transform=test_transform)
    test_loader = DataLoader(test_ds, batch_size=4, shuffle=False, num_workers=4)

    results = {}

    with torch.no_grad():
        for imgs, names, (orig_hs, orig_ws) in tqdm(test_loader):
            imgs = imgs.to(DEVICE)

            # TTA: 좌우 반전만 적용 (안전하고 확실한 성능 향상)
            # 멀티 스케일 리사이징은 안 씁니다. (점수 하락 방지)
            with autocast('cuda', dtype=torch.bfloat16):
                p1 = torch.sigmoid(model(imgs))
                p2 = torch.sigmoid(model(torch.flip(imgs, dims=[3])))
                preds = (p1 + torch.flip(p2, dims=[3])) / 2

            preds = preds.float().cpu().numpy()

            for i, name in enumerate(names):
                orig_h = orig_hs[i].item()
                orig_w = orig_ws[i].item()

                poly_str = preds_to_polygons(preds[i][0], orig_h, orig_w)
                results[name] = poly_str

    sample_df = pd.read_csv(SAMPLE_SUB)
    sample_df['polygons'] = sample_df['filename'].map(results).fillna("")
    sample_df.to_csv(OUTPUT_CSV, index=False)
    print(f"🏆 SOTA 도전 파일 생성 완료: {OUTPUT_CSV}")
    print("👉 이 파일이 98점을 찍을 주인공입니다.")

if __name__ == "__main__":
    main()

🔥 최종 추론 시작! (Model: tu-convnext_large, Unclip: 3.0)


100%|██████████| 104/104 [00:33<00:00,  3.12it/s]


🏆 SOTA 도전 파일 생성 완료: submission_pseudo_sota.csv
👉 이 파일이 98점을 찍을 주인공입니다.


In [10]:
import os
import json
import cv2
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from torch.amp import autocast
import albumentations as A
from albumentations.pytorch import ToTensorV2
import segmentation_models_pytorch as smp
import pyclipper
from shapely.geometry import Polygon
from tqdm import tqdm
import ssl
import warnings

warnings.filterwarnings('ignore')
ssl._create_default_https_context = ssl._create_unverified_context

# --- [최고 점수 0.9553점 확정 세팅] ---
ENCODER_NAME = "tu-convnext_base"
# 아까 저장해둔 모델 파일명으로 맞춰주세요!
MODEL_PATH = "DBNet_ConvNeXt_FPN_final.pth"
OUTPUT_CSV = "final_submission_0.9553.csv"

# 핵심 파라미터 (이게 점수의 비결입니다)
UNCLIP_RATIO = 3.0   # 통통하게 불리기
BOX_THRESH = 0.3     # 과감하게 잡기

BASE_PATH = './data/datasets'
TEST_IMG_DIR = os.path.join(BASE_PATH, 'images/test')
TEST_JSON = os.path.join(BASE_PATH, 'jsons/test.json')
SAMPLE_SUB = os.path.join(BASE_PATH, 'sample_submission.csv')

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
IMG_SIZE = 1024

class TestDataset(Dataset):
    def __init__(self, img_dir, json_path, transform=None):
        self.img_dir = img_dir
        self.transform = transform
        with open(json_path, 'r', encoding='utf-8') as f:
            self.data = json.load(f)['images']
        self.image_names = list(self.data.keys())

    def __len__(self): return len(self.image_names)

    def __getitem__(self, idx):
        name = self.image_names[idx]
        image = cv2.imread(os.path.join(self.img_dir, name))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        orig_h, orig_w = image.shape[:2]
        if self.transform:
            image = self.transform(image=image)['image']
        return image, name, (orig_h, orig_w)

def unclip(box, unclip_ratio):
    poly = Polygon(box)
    if poly.area <= 0 or poly.length <= 0: return box
    distance = poly.area * unclip_ratio / poly.length
    offset = pyclipper.PyclipperOffset()
    offset.AddPath(box, pyclipper.JT_ROUND, pyclipper.ET_CLOSEDPOLYGON)
    expanded = offset.Execute(distance)
    return expanded

def preds_to_polygons(pred_mask, orig_h, orig_w):
    pred_mask = cv2.resize(pred_mask, (orig_w, orig_h))
    binary_mask = (pred_mask > BOX_THRESH).astype(np.uint8)
    contours, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    polygons = []
    for cnt in contours:
        if cv2.contourArea(cnt) < 30: continue
        epsilon = 0.003 * cv2.arcLength(cnt, True)
        approx = cv2.approxPolyDP(cnt, epsilon, True)
        if len(approx) < 3: continue

        reshaped_poly = approx.reshape(-1, 2)
        try:
            expanded = unclip(reshaped_poly, UNCLIP_RATIO)
            if len(expanded) > 0:
                final_poly = np.array(expanded[0])
                polygons.append(final_poly.reshape(-1).tolist())
            else:
                polygons.append(reshaped_poly.reshape(-1).tolist())
        except:
            polygons.append(reshaped_poly.reshape(-1).tolist())

    return "|".join([" ".join(map(str, poly)) for poly in polygons])

def main():
    print(f"🔥 추론 시작: Single Scale (Unclip {UNCLIP_RATIO})")

    model = smp.FPN(encoder_name=ENCODER_NAME, in_channels=3, classes=1).to(DEVICE)

    if os.path.exists(MODEL_PATH):
        model.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE))
    elif os.path.exists("DBNet_ConvNeXt_FPN_best.pth"): # best 파일명 대응
        model.load_state_dict(torch.load("DBNet_ConvNeXt_FPN_best.pth", map_location=DEVICE))
    else:
        print("❌ 모델 파일이 없습니다!")
        return

    model.eval()

    test_transform = A.Compose([
        A.Resize(IMG_SIZE, IMG_SIZE),
        A.Normalize(),
        ToTensorV2()
    ])

    test_ds = TestDataset(TEST_IMG_DIR, TEST_JSON, transform=test_transform)
    test_loader = DataLoader(test_ds, batch_size=8, shuffle=False, num_workers=4)

    results = {}

    with torch.no_grad():
        for imgs, names, (orig_hs, orig_ws) in tqdm(test_loader):
            imgs = imgs.to(DEVICE)

            # TTA: 좌우 반전(Flip)만 사용 (이건 점수 높이는 데 도움 됨)
            with autocast('cuda', dtype=torch.bfloat16):
                p1 = torch.sigmoid(model(imgs))
                p2 = torch.sigmoid(model(torch.flip(imgs, dims=[3])))
                preds = (p1 + torch.flip(p2, dims=[3])) / 2

            preds = preds.float().cpu().numpy()

            for i, name in enumerate(names):
                orig_h = orig_hs[i].item()
                orig_w = orig_ws[i].item()
                poly_str = preds_to_polygons(preds[i][0], orig_h, orig_w)
                results[name] = poly_str

    sample_df = pd.read_csv(SAMPLE_SUB)
    sample_df['polygons'] = sample_df['filename'].map(results).fillna("")
    sample_df.to_csv(OUTPUT_CSV, index=False)
    print(f"🎉 파일 생성 완료: {OUTPUT_CSV}")

if __name__ == "__main__":
    main()

🔥 추론 시작: Single Scale (Unclip 3.0)


100%|██████████| 52/52 [00:29<00:00,  1.76it/s]


🎉 파일 생성 완료: final_submission_0.9553.csv
